## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

##### 可以自行嘗試多種架構

In [2]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input
 
from tensorflow.keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
#model=tensorflow.keras.applications.xception.Xception(include_top=False, weights='imagenet',
                                             #input_tensor=input_tensor,pooling=None, classes=2)
"""自行填入"""

'''Resnet 50 架構'''
model=tensorflow.keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None, classes=10)
model.summary()


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [3]:
config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))
config.gpu_options.per_process_gpu_memory_fraction = 0.85  #占用85%显存
session = tf.compat.v1.Session(config=config)

## 添加層數

In [4]:
x = model.output
x=Flatten()(x)
'''可以參考Cifar10實作章節,自行填入'''
x = Dense(1000, activation='relu')(x)
x = Dense(1000, activation='relu')(x)
predictions = Dense(10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 179


## 鎖定特定幾層不要更新權重

In [5]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
500/500 [==============================] - 29s 33ms/step - loss: 1.9523 - accuracy: 0.3343
Epoch 2/100
500/500 [==============================] - 17s 33ms/step - loss: 1.3489 - accuracy: 0.5294
Epoch 3/100
500/500 [==============================] - 17s 33ms/step - loss: 1.2208 - accuracy: 0.5716
Epoch 4/100
500/500 [==============================] - 16s 33ms/step - loss: 1.1122 - accuracy: 0.6104
Epoch 5/100
500/500 [==============================] - 17s 33ms/step - loss: 1.0200 - accuracy: 0.6414
Epoch 6/100
500/500 [==============================] - 17s 33ms/step - loss: 0.9267 - accuracy: 0.6725
Epoch 7/100
500/500 [==============================] - 17s 33ms/step - loss: 0.8423 - accuracy: 0.6998
Epoch 8/100
500/500 [==============================] - 17s 33ms/step - loss: 0.7449 - accuracy: 0.7381
Epoch 9/100
500/500 [==============================] - 17s 34ms/step - loss: 0.6538 - accuracy: 0.7642
Epoch 10/100
500/500 [==============================] - 17s 34ms/step - l